## Sources and details of the dataset

Foursquare is a social media platform where users can share their locations and rate venues. This dataset contains 09/2013's values.

You need to download the dataset with torrent to access it.

> https://archive.org/download/201309_foursquare_dataset_umn/201309_foursquare_dataset_umn_archive.torrent

<img src="images/raw_dataset.png">

### [Content of Files](https://archive.org/details/201309_foursquare_dataset_umn)

 * users.dat: Consists of a set of users such that each user has a unique id and a geospatial location (latitude and longitude) that represents the user home town location.
 * venues.dat: Consists of a set of venues (e.g., restaurants) such that each venue has a unique id and a geospatial location (lattude and longitude).
 * checkins.dat: Marks the checkins (visits) of users at venues. Each check-in has a unique id as well as the user id and the venue id.
 * socialgraph.dat: Contains the social graph edges (connections) that exist between users. Each social connection consits of two users (friends) represented by two unique ids (first_user_id and second_user_id).
 * ratings.dat: Consists of implicit ratings that quantifies how much a user likes a specific venue.

## A proposition on what, why and how to work with the data

We don't have a clear hypothesis beforehand we will conduct exploratory analysis.

### How we will work with foursquare dataset?

  * Download dataset from source with torrent
  * Clean and transform it with python
  * Import it to PostgreSQL and design schema, tables and indexing for performance improvements.
  * Analyze it with sql queries
  * Visualize the results with python(i.e. matplotlib)
  * Share the report and results with ipython notebook

## Outline of the report

* Introduction and details of the dataset.
* Basic descriptive analysis of each table.
* Social network analysis.
* Reviewers and review activity analysis.
* Venue popularity and rating analysis.

## Accessing data from file system and cleaning it

In [6]:
import time
import math
import os
import re
import filecmp

source_dir = os.getcwd()
raw_data_folder = source_dir+"\\data\\raw"
clean_data_folder = source_dir+"\\data\\clean"

for filename in filecmp.dircmp(raw_data_folder, clean_data_folder).common:
    print("{filename} already exists.".format(filename=filename))

for filename in filecmp.dircmp(raw_data_folder, clean_data_folder).left_only: #this prevents processing already cleaned data, swap with below comment if you want to process it anyway
#for filename in os.listdir(raw_data_folder):
    with open(os.path.join(raw_data_folder, filename), 'r') as f: #open in readonly mode
        start_time = math.trunc(time.time())
        clean_file = open(os.path.join(clean_data_folder,filename),"w")
        line_number = 0
        for line in f:
            line_number += 1 #Don't carry this to end of the elif statements otherwise line number will be zero indefinitely
            if(line_number in range(1,3)):
                #print("Deleted line: {line_number}".format(line_number=line_number))
                continue
            elif(line.endswith("rows)\n")):
                break
            else:
                clean_format = line.replace(" ","").replace("|",";")
                clean_file.write(clean_format)
        clean_file.close()
        end_time = math.trunc(time.time())
        print("Cleaned and created {filename} in {execute_time} seconds".format(execute_time=end_time-start_time, filename=filename))

checkins.dat already exists.
ratings.dat already exists.
socialgraph.dat already exists.
users.dat already exists.
venues.dat already exists.


In [7]:
import psycopg2

pghost = "localhost"
pguser = "postgres"
pgdatabase = "MEF-BDA-PROD"
pgport = "5432"
pgpassword = "123"

conn_string = 'host={pghost} port={pgport} dbname={pgdatabase} user={pguser} password={pgpassword}'.format(pgdatabase=pgdatabase,pguser=pguser,pgpassword=pgpassword,pghost=pghost,pgport=pgport)
conn=psycopg2.connect(conn_string)
cur=conn.cursor()

def check_if_table_exists(schema,table):
    cur.execute("select exists(select * from information_schema.tables where table_schema='{schema}' AND table_name='{table}')".format(schema=schema, table=table))
    return cur.fetchone()[0]

def check_if_index_exists(index):
    cur.execute("SELECT EXISTS(SELECT * FROM PG_CLASS WHERE relname = '{index}')".format(index=index))
    return cur.fetchone()[0]

if(check_if_table_exists('ODS','EXT_FS_USERS')):
    print('Table ODS.EXT_FS_USERS already exists.')   
else:
    start_time = math.trunc(time.time())
    cur.execute("""
    CREATE TABLE "ODS"."EXT_FS_USERS"
    (
    id integer,
    latitude double precision,
    longitude double precision
    )

    TABLESPACE pg_default;

    ALTER TABLE "ODS"."EXT_FS_USERS"
    OWNER to postgres;
    """)
    end_time = math.trunc(time.time())
    cur.execute('COMMIT;')
    print("Table ODS.EXT_FS_USERS created in {execute_time} seconds.".format(execute_time=end_time-start_time))
    
    #start_time = math.trunc(time.time())
    #cmd_command = """"C:\\Program Files\\PostgreSQL\\13\\bin\\psql.exe" -h {pghost} -U {pguser} -d {pgdatabase} -p {pgport};
    #{pgpassword}; 
    #\COPY "ODS"."EXT_FS_VENUES" FROM '{datasource}' WITH (FORMAT CSV, DELIMITER ';');
    #""".format(pgdatabase=pgdatabase,pguser=pguser,pgpassword=pgpassword,pghost=pghost,pgport=pgport,datasource=clean_data_folder.replace('\\','/')+"/users.dat")
    #print(cmd_command)
    #os.system('cmd /k {cmd_command}'.format(cmd_command=cmd_command))
    #end_time = math.trunc(time.time())
    #print("Imported data to ODS.EXT_FS_USERS in {execute_time} seconds.".format(execute_time=end_time-start_time))

if(check_if_table_exists('ODS','EXT_FS_VENUES')):
    print('Table ODS.EXT_FS_VENUES already exists.')   
else:
    start_time = math.trunc(time.time())
    cur.execute("""
    CREATE TABLE "ODS"."EXT_FS_VENUES" AS 
    (
    id integer,
    latitude double precision,
    longitude double precision
    )
    
    TABLESPACE pg_default;

    ALTER TABLE "ODS"."EXT_FS_VENUES"
    OWNER to postgres;
    """)
    cur.execute('COMMIT;')
    end_time = math.trunc(time.time())
    print("Table ODS.EXT_FS_VENUES created in {execute_time} seconds.".format(execute_time=end_time-start_time))

Table ODS.EXT_FS_USERS already exists.
Table ODS.EXT_FS_VENUES already exists.


In [5]:
cur.close()
conn.close()

## Sources:

 1. [Dataset](https://archive.org/details/201309_foursquare_dataset_umn)

 2. [How to open every file in a folder?](https://stackoverflow.com/questions/18262293/how-to-open-every-file-in-a-folder)

 3. [Python- reading a file line by line and processing](https://stackoverflow.com/questions/53749062/python-reading-a-file-line-by-line-and-processing)

 4. [File and Directory Comparisons with Python](https://janakiev.com/blog/python-filecmp/)

 5. [Checking if a table exist with psycopg2 on postgreSQL](https://stackoverflow.com/questions/1874113/checking-if-a-postgresql-table-exists-under-python-and-probably-psycopg2)

 6. [Checking if index exist](https://stackoverflow.com/questions/45983169/checking-for-existence-of-index-in-postgresql)